In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
import scipy.stats as stats
import seaborn as sns

plt.style.use("ggplot")
sns.set_context("poster")

ハイパーパラメーターを設定する。簡単のために、全クラスターで同じハイパーパラメーターを採用する。

In [3]:
# Parameters of * distribution
Gam = { "a": [0.0], "b": 0.0 }
Poi = { "lambda": [0.0] }

# Bayesian Poisson Mixture Model 
BPMM = { "D": 0, "K": 0, "alpha": [0.0], "cmp": {Gam} }
# Poisson Mixture Model 
PMM = { "D": 0, "K": 0, "phi": [0.0], "cmp": {Poi}}

SyntaxError: invalid syntax (<ipython-input-3-853d69bfd3e4>, line 1)

In [ ]:
## Common functions

# Sample a PMM given hyperparameters.
function sample_PMM(bpmm::BPMM)
    cmp = Vector{Poi}()
    for c in bpmm.cmp
        lambda = Vector{Float64}()
        for d in 1 : bpmm.D
            push!(lambda, rand(Gamma(c.a[d], 1.0/c.b)))
        end
        push!(cmp, Poi(lambda))  
    end
    phi = rand(Dirichlet(bpmm.alpha))
    return PMM(bpmm.D, bpmm.K, phi, cmp)
end

# Sample data from a specific PMM model.
function sample_data(pmm::PMM, N::Int)
    X = zeros(pmm.D, N)
    S = categorical_sample(pmm.phi, N)
    for n in 1 : N
        k = indmax(S[:, n])
        for d in 1 : pmm.D
            X[d,n] = rand(Poisson(pmm.cmp[k].lambda[d]))
        end
    end
    return X, S
end

categorical_sample(p::Vector{Float64}) = categorical_sample(p, 1)[:,1]
function categorical_sample(p::Vector{Float64}, N::Int)
    K = length(p)
    S = zeros(K, N)
    S_tmp = rand(Categorical(p), N)
    for k in 1 : K
        S[k,find(S_tmp.==k)] = 1
    end
    return S
end

function init_S(X::Matrix{Float64}, bpmm::BPMM)
    N = size(X, 2)
    K = bpmm.K
    S = categorical_sample(ones(K)/K, N)    
    return S
end

In [ ]:
s_n = stats.